<a href="https://colab.research.google.com/github/neklyud/pytorch-tutorial/blob/master/pytorch_tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

In [0]:
x = torch.ones(2,2,requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [0]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [0]:
print(y.grad_fn)

In [0]:
z = y*y*3
out = z.mean()
print(out)

IndexError: ignored

In [0]:
a = torch.randn(2,2)
a = ((a*3)/(a-1))
print(a.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)

b = (a*a).sum()
print(b.grad_fn)

False
True


In [0]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [0]:
x = torch.randn(3, requires_grad = True)
print(x)
y = x*2
print(y)
print(y.data.norm())
while y.data.norm() < 1000:
  y = y*2

print(y)

tensor([-0.1474, -0.1420, -0.2002], requires_grad=True)
tensor([-0.2947, -0.2840, -0.4004], grad_fn=<MulBackward0>)
tensor(0.5726)
tensor([-603.5518, -581.6725, -820.0197], grad_fn=<MulBackward0>)


In [0]:
v = torch.tensor([0.1,1.0,0.0001], dtype=torch.float)
y.backward(v)
print(x.grad)

RuntimeError: ignored

In [0]:
print(x.requires_grad)
print((x*2).requires_grad)

with torch.no_grad():
  print((x*2).requires_grad)


SyntaxError: ignored

In [0]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 канал ввода изображения, 
        # 6 каналов вывода, 
        # 5x5 квадратное сверточное ядро
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # аффинная операция: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Максимальное объединение через (2, 2) окно
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # Если размер - это квадрат,
        # тогда вы можете задать только одно число
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        # все измерения, исключая измерение пакета
        size = x.size()[1:]  
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [0]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params)

In [0]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0911, -0.0996,  0.1200, -0.0841, -0.0358,  0.0782, -0.0851, -0.0751,
         -0.0693, -0.0350]], grad_fn=<AddmmBackward>)


In [0]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [0]:
output = net(input)
target = torch.randn(10)  # случайный target, например
target = target.view(1, -1)  # сделаем его такой же формы как вывод
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.3410, grad_fn=<MseLossBackward>)


In [0]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [0]:
# обнуляем градиентные буферы всех параметров
net.zero_grad()  

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


RuntimeError: ignored

In [0]:
params = list(net.parameters())
print(len(params))
print(params)

0
[]
